In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from datetime import datetime

In [2]:
spark = SparkSession.\
        builder.\
        master('local[*]').\
        appName('hw_2023').\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/13 08:10:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 08:10:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/13 08:10:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Question 1: Execute spark.version, what is the version?

spark.version

#### Answer: 3.3.2

'3.3.2'

In [4]:
# Question 2: HVFHW June 2021

# Read it with Spark using the same schema as we did in the lessons.
# We will use this dataset for all the remaining questions.
# Repartition it to 12 partitions and save it to parquet.
# What is the average size of the Parquet
# (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz -o 2023.csv.gz
!gzip -d 2023.csv.gz
csv_df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('2023.csv')
csv_df.repartition(12).write.parquet('hw_2023/')
!rm -f 2023/.par*
!ls -lash hw_2023/

### Answer: 23

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  167M  100  167M    0     0  74.6M      0  0:00:02  0:00:02 --:--:-- 98.3M


[Stage 4:====================================================>    (11 + 1) / 12]

total 273M
4.0K drwxr-xr-x 2 ajay ajay 4.0K Jan 13 08:12 .
4.0K drwxrwxr-x 4 ajay ajay 4.0K Jan 13 08:11 ..
4.0K -rw-r--r-- 1 ajay ajay    8 Jan 13 08:12 ._SUCCESS.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00000-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00001-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00002-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00003-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00004-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00005-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 08:11 .part-00006-76ee30e8-ac5d-43eb-8c92-88e2867279c6-c000.snappy.parquet.crc

In [6]:
# Question 3: Count records
# How many taxi trips were there on June 15?
# Consider only trips that started on June 15.

df = spark.read.parquet('hw_2023/*')
df.createTempView('hw')
spark.sql(("""
            
            select 
                count(*) as num_trips 
            from hw
            where date(pickup_datetime) = '2021-06-15'
            
            """)).show()

#### Answer: 452470

[Stage 6:>                                                          (0 + 4) / 4]

+---------+
|num_trips|
+---------+
|   452470|
+---------+



In [7]:
# Question 4: Longest trip for each day
# Now calculate the duration for each trip. How long was the longest trip in Hours?

spark.sql(("""
           
            select 
                date(pickup_datetime) as date, 
                round(max(
                    (unix_timestamp(dropoff_datetime) - 
                    unix_timestamp(pickup_datetime)) 
                    / 3600.0), 2) as longest
            from hw
            group by 1
            order by 2 desc
            limit 5
            
            """)).show()

#### Answer: 66.88

[Stage 9:============================================>              (3 + 1) / 4]

+----------+-------+
|      date|longest|
+----------+-------+
|2021-06-25|  66.88|
|2021-06-22|  25.55|
|2021-06-27|  19.98|
|2021-06-26|  18.20|
|2021-06-23|  16.47|
+----------+-------+



In [8]:
# Question 5: User Interface
# Spark’s User Interface which shows application's dashboard runs on which local port?

spark._activeSession

### Answer: currently running on 4042 but usally on 4040

In [9]:
# Question 6: Most frequent pickup location zone
# Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -o z.csv
zone = spark.read.option('header', 'true').option('inferSchema','true').csv('z.csv')
joined = df.join(zone, zone.LocationID == df.PULocationID, 'left')
joined.createOrReplaceTempView('hw')
spark.sql(("""
            select 
                Zone
            from hw
            group by Zone
            order by count(*) desc
            """)).limit(5).show()

#### Answer: Crown Heights North

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  22605      0 --:--:-- --:--:-- --:--:-- 22605


[Stage 15:==============>                                           (1 + 3) / 4]

+-------------------+
|               Zone|
+-------------------+
|Crown Heights North|
|       East Village|
|        JFK Airport|
|     Bushwick South|
|      East New York|
+-------------------+



In [10]:
spark.stop()
!rm -rf *.csv *.parquet hw_*